In [95]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re 

In [100]:
url='https://lawsnote.com/search/indictment/%E5%82%B7%E5%AE%B3'
base_url='https://lawsnote.com'
driver=webdriver.Chrome()
driver.get(url)
try:
    element=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "search-result__main")))
except:
    print('error')
    driver.close()
driver=login(driver)
html=driver.page_source
soup=bs(html,'html.parser')
sub_item=catch_href(soup) #抓出頁面中的判決書連結
indictment=search_fit_indictment(sub_item) #篩出府和條件的連結

In [101]:
driver.get('https://lawsnote.com/indictment/5c12cb943e6d28c46a53367f?t=1546689104')

In [102]:
soup=bs(driver.page_source,'lxml')
#起訴書處理title
title_label=soup.select('.document__section.document__meta-content')
title_text=title_label[0].find_all('li')
judgement={}
judgement['字號']=title_text[0].text.replace('【字號】','')
judgement['日期']=title_text[1].text.replace('【日期】','')
judgement['案由']=title_text[2].text.replace('【案由】','')
#起訴書處理內文 content_label
content_label=soup.select('.document__section.document__judgement-content')
judgement['起訴書內文']=content_label[0].text
#起訴書相關法條
related_Issue=soup.select('.document-misc__sub-container.document-misc__relatedIssue-container')
r=related_Issue[0].select('.document-misc__item-container.indictment-misc-content')
#起訴書相關法條標題 related_Issue_title
related_Issue_title=[x.find('div').text for x in r]
#起訴書相關法條內文  {法條名:內文} related_Issue_content
related_Issue_content={}
for i in range(len(related_Issue_title)):
    related_Issue_content[related_Issue_title[i]]=r[i].find_all('ul')[0].text

#起訴書dict key:字號,日期,案由,起訴書內文,相關法條標題:list,相關法條內文:{法條名:內文},判決書
judgement['相關法條']=related_Issue_title
judgement['法條內文']=related_Issue_content
judg=soup.select('.document-misc__sub-container.document-misc__judgementHistory-container')
judgement['判決書']=judg[0].find('a')['id']

In [104]:
judg=soup.select('.document-misc__sub-container.document-misc__judgementHistory-container')
judg[0].find('a')['id']
driver.find_element_by_link_text(judg[0].find('a')['id']).click()



In [105]:
soup=bs(driver.page_source,'lxml')
#判決書處理title
j_t_label=soup.select('.document__section.document__meta-content')
j_t_text=j_t_label[0].find_all('li')
j_judgement={}
j_judgement['字號']=title_text[0].text.replace('【裁判字號】','')
j_judgement['日期']=title_text[1].text.replace('【裁判日期】','')
j_judgement['案由']=title_text[2].text.replace('【裁判案由】','')
#判決書內文 judgment_text
judgment_text=soup.select('.document__section.document__judgement-content.markerarea-main')[0].text
j_judgement['裁判內文']=judgment_text
#判決書相關法條
j_related_Issue=soup.select('.document-misc__sub-container.document-misc__relatedIssue-container')
j=j_related_Issue[0].select('.document-misc__item-container.judgement-misc-content')
#判決書相關法條標題 j_rel_Issue_title
j_rel_Issue_title=[x.find('div').text for x in j]
#判決書相關法條內文  {法條名:內文} j_rel_Issue_content
j_rel_Issue_content={}
for i in range(len(j_rel_Issue_title)):
    j_rel_Issue_content[j_rel_Issue_title[i]]=j[i].find_all('ul')[0].text

#判決書dict key:字號,日期,案由,裁判內文,相關法條標題:list,相關法條內文:{法條名:內文}
j_judgement['相關法條']=j_rel_Issue_title
j_judgement['法條內文']=j_rel_Issue_content



In [ ]:
fail_open=[]
i=0;
while len(indictment)>0:
    driver.find_element_by_link_text(indictment[i]).click()
    try:
        element=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'main-view')))
    except:
        fail_open.append(indictment[i])
        driver.back()
    #parse
    indictment.remove(indictment[i])


In [96]:
def catch_href(soup):
    sub_item=soup.find_all('a',{'class':'search-result__item-link'})
    return sub_item

In [97]:
def search_fit_indictment(sub_item):
    indictment_item=[]
    for s in sub_item:
        if s.text.find('傷害')!=-1 and s.text.find('起訴書')!=-1:
            indictment_item.append(s.text)
    return indictment_item

In [98]:
def login(driver):
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='揪團'])[1]/preceding::button[1]").click()
    driver.find_element_by_id("email").click()
    driver.find_element_by_id("email").clear()
    driver.find_element_by_id("email").send_keys("scott01272001@gmail.com")
    driver.find_element_by_id("password").clear()
    driver.find_element_by_id("password").send_keys("asd753")
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='忘記密碼？'])[1]/following::button[1]").click()
    return driver